<a href="https://colab.research.google.com/github/varshini2308/Internship-Agent/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gtts
from gtts import gTTS
from IPython.display import Audio, display
import requests


In [3]:
import os
import requests
from gtts import gTTS
from IPython.display import Audio, display
import threading  # Import threading module for managing audio playback in a separate thread

# Set the API key as an environment variable
os.environ["TOGETHER_API_KEY"] = "e72f558a409a0ac55156ce43b4fc41d03c04e25ec6c5ae7c54abed8692b69d07"

# Global variable to track audio playback
stop_audio = threading.Event()

def get_llama_response(question):
    url = "https://api.together.xyz/v1/chat/completions"
    api_key = os.environ.get("TOGETHER_API_KEY")

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "WizardLM/WizardLM-13B-V1.2",
        "messages": [{"role": "user", "content": question}],
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        try:
            data = response.json()
            choices = data.get("choices", [])
            if choices:
                message = choices[0].get("message", {}).get("content", "")
                return message
            else:
                return "No response from the API"
        except Exception as e:
            return f"Error parsing JSON: {e}"
    else:
        return f"Error: {response.status_code} - {response.text}"

def read_aloud(text):
    global stop_audio  # Use global variable to track audio playback

    stop_audio.clear()  # Clear the stop event before starting audio playback
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    audio_widget = Audio("response.mp3", autoplay=True)
    display(audio_widget)

    # Use a separate thread to manage audio playback
    def audio_thread():
      while True:
        if stop_audio.is_set():  # Check if stop event is set
            stop_audio.clear()  # Clear the stop event to stop audio playback
            break


    threading.Thread(target=audio_thread).start()  # Start the audio playback thread

def handle_multiple_questions():
    while True:
        user_question = input("What's your question? (Type 'exit' to quit): ")
        if user_question.lower() == 'exit':
            print("Session ended.")
            break

        response = get_llama_response(user_question)
        print("Llama's response:", response)
        read_aloud(response)

        stop_audio_input = input("Do you want to ask a new question? (y/n): ")
        if stop_audio_input.lower() == 'y':
            stop_audio.set()
        elif stop_audio_input.lower() == 'n':
            print("Session ended. Thank you and have a great day!!")
            break

# Start the session to handle multiple questions
handle_multiple_questions()

What's your question? (Type 'exit' to quit): hello
Llama's response:   Hello! How can I help you today?


Do you want to ask a new question? (y/n): y
What's your question? (Type 'exit' to quit): who is the prime minister of India?
Llama's response:   As of my knowledge cutoff date in September 2021, the Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister of India since May 2014. However, there may have been changes since then, so it's always a good idea to check the most recent updates.


Do you want to ask a new question? (y/n): n
Session ended. Thank you and have a great day!!
